TODO: i have x number of midi files, associate each midi file to genre somehow. then it is possible to create tokenizer file which contains prompt of genre and tokenized midi file.

In [55]:
import os

def get_folder_names(directory):
    # List all entries in the directory
    entries = os.listdir(directory)
    
    # Filter out only directories
    folder_names = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
    
    return folder_names

# Example usage
directory_path = './data'
folders = get_folder_names(directory_path)
print(folders)

['VOF_de_Kunst', 'Eubie_Blake', 'Wilson,Jackie', 'The_Escape_Club', 'Morricone_Ennio', 'Honey_Cone', 'The_Four_Aces', 'Vaughn', 'Cocciante', 'Mr._Mister', 'Julian_Lennon', 'Del_Shannon', 'Dick_Hyman', 'Thompson_Twins', 'LIGABUE', 'Al_Stewart', 'Billie_Myers', 'The_Brian_Setzer_Orchestra', 'Metallica', 'Travis_Tritt', 'Johnny_Logan', 'Traffic', 'Peter_Gabriel', 'Inner_City', 'Ted_Nugent', 'Christie_Lou', 'Gerry_Rafferty', 'Alunni_del_Sole', 'The_Easybeats', 'EMF', 'Carly_Simon', 'Valens_Ritchie', 'Wendy_Moten', 'Chesney_Hawkes', 'Mo-Do', 'Weather_Report', 'Sheb_Wooley', 'Haggard._Merle', 'U96', 'Air_Supply', 'Battisti_Lucio', 'Jones', 'Ron', 'Don_Henley', 'The_Verve_Pipe', 'Blackstreet', 'Westbam', 'R._Kelly', 'Bertie_Higgins', 'Traveling_Wilburys', 'Gianni_Bella', 'Irene_Cara', 'Master_P', 'Amanda_McBroom', 'The_Lemon_Pipers', 'Renato_Teixeira', 'The_Lemonheads', 'Sonny_and_Cher', 'Swing_Out_Sister', 'The_Soul_Survivors', 'Ellington', 'U.S.A._for_Africa', 'Randy_Newman', 'En_Vogue', 'M

In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Replace these with your own Spotify API credentials:
client_id = '78e0f8845bec4608a69e261379ce79de'
client_secret = '46286433e2ac48ba8b5a1795920d209f'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_genres(artist_name):
    results = sp.search(q='artist:' + artist_name, type='artist')
    artists = results['artists']['items']
    for artist in artists:
        print("Name:", artist.get('name'))
        print("Genres:", artist.get('genres'))
        print("---")
    # Optionally, return the genres for the first matching artist that has non-empty genres
    for artist in artists:
        if artist.get('genres'):
            return artist.get('genres')
    return []

# Example usage:
artist = "Ringo_Starr"
genres = get_artist_genres(artist)
print(f"\n{artist} genres (from first match with non-empty genres): {genres}")

Name: Ringo Starr
Genres: []
---
Name: Ringo Starrbucks
Genres: []
---
Name: Ringo Starr
Genres: []
---
Name: Las Narices de Ringo Starr
Genres: []
---

Ringo_Starr genres (from first match with non-empty genres): []


In [ ]:
broad_genre_mapping = {
    "pop": ["pop"],
    "rock": ["rock", "rockabilly", "glam rock", "hard rock", "punk", "grunge"],
    "hip hop": ["hip hop", "rap", "gangster rap", "drill", 'southern hip hop','grime'],
    "electronic": ["electronic", "edm", "house", "techno", "dubstep", "trance", "synthwave"],
    "jazz": ["jazz", "smooth jazz", "jazz fusion", "bebop", "vocal jazz",'ragtime','motown'],
    "classical": ["classical", "baroque", "chamber", "orchestra", "neoclassical"],
    "country": ["country", "honky tonk", "traditional country", "texas country"],
    "r&b": ["r&b", "soul", 'new jack swing','blues','blues rock'],
    "metal": ["metal", "black metal", "glam metal", "industrial metal", "nu metal", "deathcore", 'gothic metal','rap metal'],
    "reggae": ["reggae", "reggae rock", "dub"],
    "folk": ["folk", "singer-songwriter"],
    "latin": ["latin", "salsa", "bossa nova", "reggaeton", "latin pop"],
    "k-pop": ["k-pop", "kpop"],
    "disco": ['disco', 'nu disco','italo disco','post-disco'],
    'gospel': ['gospel', 'southern gospel']
}

In [31]:
import csv
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from requests.exceptions import ReadTimeout

# Replace with your Spotify API credentials
client_id = '78e0f8845bec4608a69e261379ce79de'
client_secret = '46286433e2ac48ba8b5a1795920d209f'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# Increase timeout by setting requests_timeout to 10 seconds
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10)

def get_artist_genre(artist_name, attempts=3):
    """
    Searches for the artist and returns the first genre if available.
    Retries the request a few times if a ReadTimeout occurs.
    """
    for attempt in range(attempts):
        try:
            results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
            items = results['artists']['items']
            if items:
                genres = items[0].get('genres', [])
                if genres:
                    return genres[0]  # Use only the first genre
            return 'Unknown'
        except ReadTimeout as e:
            print(f"ReadTimeout for {artist_name} on attempt {attempt + 1}: {e}")
            time.sleep(1)  # Pause briefly before retrying
    return 'Unknown'

with open('artist_genres1.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['artist', 'genre'])
    
    for artist in folders:
        genre = get_artist_genre(artist)
        writer.writerow([artist, genre])
        print(f"Processed: {artist} -> {genre}")
        time.sleep(0.1)  # Adjust delay if needed to avoid rate limits

Processed: Spirit -> Unknown
Processed: Axelle_Red -> variété française
Processed: Clerc_Julien -> chanson
Processed: Le_Grand_Orchestre_du_Splendid -> chanson
Processed: Fugees -> east coast hip hop
Processed: Boney_James -> smooth jazz
Processed: Ricchi_e_Poveri -> italo dance
Processed: Gilbert_Becaud -> chanson
Processed: Orgy -> Unknown
Processed: Stiltskin -> Unknown
Processed: Tears_n_Joy -> Unknown
Processed: Kevin_Paige -> Unknown
Processed: Beloved -> Unknown
Processed: Save_Ferris -> ska
Processed: Propaganda -> christian hip hop
Processed: Michael_George -> Unknown
Processed: Diamond_Jim -> Unknown
Processed: Taja_Sevelle -> Unknown
Processed: Koos_Alberts -> nederpop
Processed: The_Go-Go_s -> Unknown
Processed: Cilla_Black -> Unknown
Processed: Joni_Mitchell -> singer-songwriter
Processed: Henry_Butler -> Unknown
Processed: Neil_Young -> classic rock
Processed: Kid_Rock -> Unknown
Processed: Village_People -> disco
Processed: The_Allman_Brothers_Band -> Unknown
Processed: 

In [29]:
import pandas as pd
df = pd.read_csv('../artist_genre_mapped.csv')

In [ ]:
import pandas as pd

# Define the mapping: broad genres with associated keywords (all in lowercase)
broad_genre_mapping = {
    "pop": [
        "pop",
        "schlager",
        "doo-wop"
    ],
    "rock": [
        "rock", "rockabilly", "glam rock", "hard rock", "punk", "grunge",
        "new wave", "post-hardcore", "madchester", "midwest emo", "neue deutsche welle"
    ],
    "hip-hop": [
        "hip hop", "rap", "gangster rap", "drill", "southern hip hop", "grime",
        "g-funk", "miami bass"
    ],
    "electronic": [
        "electronic", "edm", "house", "techno", "dubstep", "trance", "synthwave",
        "eurodance", "happy hardcore", "bassline", "trip hop", "brazilian bass",
        "big beat", "downtempo", "idm", "melodic bass", "freestyle", "gabber", "uk garage"
    ],
    "jazz": [
        "jazz", "smooth jazz", "jazz fusion", "bebop", "vocal jazz", "ragtime", "motown",
        "big band", "swing music", "hard bop", "boogie-woogie"
    ],
    "classical": [
        "classical", "baroque", "chamber", "orchestra", "neoclassical",
        "opera", "choral"
    ],
    "country": [
        "country", "honky tonk", "traditional country", "texas country",
        "red dirt"
    ],
    "r&b": [
        "r&b", "soul", "new jack swing", "blues", "blues rock",
        "quiet storm", "funk", "funk melody"
    ],
    "metal": [
        "metal", "black metal", "glam metal", "industrial metal", "nu metal", "deathcore",
        "gothic metal", "rap metal"
    ],
    "reggae": [
        "reggae", "reggae rock", "dub",
        "dancehall"
    ],
    "folk": [
        "folk", "singer-songwriter",
        "americana", "bluegrass", "traditional music", "celtic", "newgrass"
    ],
    "latin": [
        "latin", "salsa", "bossa nova", "reggaeton", "latin pop",
        "bachata", "merengue", "vallenato", "corrido", "cumbia sonidera",
        "sierreño", "cumbia norteña", "duranguense", "cuarteto", "sertanejo tradicional",
        "sertanejo", "tejano", "electro corridos", "trova", "forró"
    ],
    "k-pop": [
        "k-pop", "kpop"
    ],
    "disco": [
        "disco", "nu disco", "italo disco", "post-disco",
        "italo dance", "hi-nrg"
    ],
    "gospel": [
        "gospel", "southern gospel", "ccm", "christian", "worship"
    ],
    # New keys for genres that don’t neatly fit into the above categories:
    "adult standards": ["adult standards", "easy listening"],
    "world": [
        "dangdut", "chanson", "variété française", "canzone napoletana", "agronejo",
        "calypso", "flamenco", "gnawa", "maluku", "entehno"
    ],
    "lo-fi": ["lo-fi beats", "lo-fi"],
    "holiday": ["christmas"],
    "children": ["lullaby", "children's music"],
    "video game": ["japanese vgm"],
    "chant": ["gregorian chant"],
    "j-pop": ["kayokyoku"],
}


def map_to_broad_genre(niche_genre):
    """
    Map a niche genre to a broad genre based on keyword matching.
    If no keyword is found, return the original niche genre.
    """
    niche_lower = niche_genre.lower()
    for broad, keywords in broad_genre_mapping.items():
        for keyword in keywords:
            if keyword in niche_lower:
                return broad
    return niche_genre  # If no mapping found, keep the original

# Read the CSV file with the artist and genre data.
# Apply the mapping function to the 'genre' column.
df["genre"] = df["genre"].apply(map_to_broad_genre)

# Write the mapped results to a new CSV file.
df.to_csv("artist_genre_mapped2.csv", index=False)

print("Mapping complete. Output saved to artist_genre_mapped.csv")

Mapping complete. Output saved to artist_genre_mapped.csv


In [ ]:
unique_genres = df[~df['genre'].isin(broad_genre_mapping.keys())]['genre'].unique()
print(unique_genres)

['eurodance' 'happy hardcore' 'Unknown' 'italo dance' 'adult standards'
 'dangdut' 'chanson' 'new age' 'big band' 'variété française' 'schlager'
 'bachata' 'comedy' 'lo-fi beats' 'christmas' 'polka' 'canzone napoletana'
 'trova' 'agronejo' 'opera' 'musicals' 'quiet storm' 'lullaby' 'hi-nrg'
 'banda' 'merengue' 'doo-wop' 'calypso' 'swing music' 'vallenato'
 'neue deutsche welle' "children's music" 'ccm' 'bolero' 'bluegrass'
 'son cubano' 'lo-fi' 'funk' 'dancehall' 'bassline' 'americana'
 'easy listening' 'sierreño' 'christian' 'new wave' 'corrido'
 'post-hardcore' 'cumbia sonidera' 'lounge' 'trip hop' 'japanese vgm'
 'brazilian bass' 'big beat' 'flamenco' 'gregorian chant' 'soundtrack'
 'exotica' 'newgrass' 'funk melody' 'hard bop' 'gnawa' 'madchester'
 'midwest emo' 'boogie-woogie' 'norteño' 'norteño-sax' 'cumbia norteña'
 'gospel' 'dark cabaret' 'celtic' 'maluku' 'entehno' 'downtempo'
 'duranguense' 'cuarteto' 'dansband' 'forró' 'sertanejo tradicional'
 'sertanejo' 'anime' 'traditiona

In [6]:
import os
import glob
import pandas as pd
import time
from miditok import REMI, TokenizerConfig
from symusic import Score

# Read CSV file with artist and mapped genre information.
df = pd.read_csv("../artist_genre_mapped.csv")
# Filter out artists with genre "Unknown" (case-insensitive)
df = df[df['genre'].str.lower() != "unknown"]

# Create a dictionary mapping from artist name to genre.
artist_to_genre = dict(zip(df['artist'], df['genre']))

In [13]:
# Define the base input folder (with artist subfolders) and the output token file.
input_base = "../data"
output_file = "all_tokenized_miditok.txt"

# Set up the MidiTok REMI tokenizer.
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)

# Count total number of MIDI files to process (for progress tracking).
total_files = 0
for artist in artist_to_genre.keys():
    artist_folder = os.path.join(input_base, artist)
    if os.path.isdir(artist_folder):
        midi_files = glob.glob(os.path.join(artist_folder, "*.mid")) + glob.glob(os.path.join(artist_folder, "*.midi"))
        total_files += len(midi_files)
print(f"Total MIDI files to process: {total_files}")

start_time = time.time()
count = 0

# Open the output file for writing tokenized sequences.
with open(output_file, "w", encoding="utf-8") as out_f:
    # Loop over each artist (whose folder name should match the CSV).
    for artist, genre in artist_to_genre.items():
        artist_folder = os.path.join(input_base, artist)
        if not os.path.isdir(artist_folder):
            print(f"Folder for artist '{artist}' not found in {input_base}. Skipping.")
            continue

        # Get all MIDI files (.mid and .midi) in the artist folder.
        midi_files = glob.glob(os.path.join(artist_folder, "*.mid")) + glob.glob(os.path.join(artist_folder, "*.midi"))
        for midi_file in midi_files:
            try:
                # Use symusic to load the MIDI file.
                score = Score(midi_file)
                # Tokenize the MIDI file using MidiTok (REMI). This returns a TokSequence object.
                tokens = tokenizer(score)
                # Convert the TokSequence to a list so we can prepend our genre token.
                tokens = list(tokens)
                
                # Prepend a genre conditioning token.
                genre_token = f"Genre_{genre}"
                tokens = [genre_token] + tokens
                
                # Convert all tokens to strings and join them with spaces.
                token_str = " ".join(str(token) for token in tokens)
                # Write the tokenized sequence to the output file (one sequence per line).
                out_f.write(token_str + "\n")
                
                count += 1
                # Print progress every 10 files or when finished.
                if count % 10 == 0 or count == total_files:
                    elapsed = time.time() - start_time
                    print(f"Processed {count}/{total_files} files. Elapsed time: {elapsed:.2f} seconds.")
            except Exception as e:
                print(f"Error processing {midi_file}: {e}")

print(f"Tokenization complete. Processed {count} files. Output saved to {output_file}.")

/Users/adityaarun/anaconda3/envs/music/lib/python3.9/site-packages/miditok/tokenizations/remi.py:88: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


Total MIDI files to process: 14047
Processed 10/14047 files. Elapsed time: 0.90 seconds.
Processed 20/14047 files. Elapsed time: 1.94 seconds.
Processed 30/14047 files. Elapsed time: 2.78 seconds.
Processed 40/14047 files. Elapsed time: 3.78 seconds.
Processed 50/14047 files. Elapsed time: 4.66 seconds.
Processed 60/14047 files. Elapsed time: 5.39 seconds.
Processed 70/14047 files. Elapsed time: 6.35 seconds.
Processed 80/14047 files. Elapsed time: 7.31 seconds.
Processed 90/14047 files. Elapsed time: 8.37 seconds.
Processed 100/14047 files. Elapsed time: 9.40 seconds.
Processed 110/14047 files. Elapsed time: 10.26 seconds.
Processed 120/14047 files. Elapsed time: 11.21 seconds.
Processed 130/14047 files. Elapsed time: 12.05 seconds.
Processed 140/14047 files. Elapsed time: 12.86 seconds.
Processed 150/14047 files. Elapsed time: 13.70 seconds.
Processed 160/14047 files. Elapsed time: 14.68 seconds.
Processed 170/14047 files. Elapsed time: 15.44 seconds.
Processed 180/14047 files. Elaps